# Preperation
- Powerpoint
- This Notebook
- Clear all output & reset
- Reset area threshold for squares with 80
- Remove 2022 from console.log
- Execute LevenshteinDistance
- Execute and clear dependences
- Start PPT
- Start Rise & F11 (Fullscreen)
- Switch to PPT

In [1]:
// https://gist.github.com/Davidblkx/e12ab0bb2aff7fd8072632b396538560

public static class LevenshteinDistance
{
    /// <summary>
    /// Calculate the difference between 2 strings using the Levenshtein distance algorithm
    /// </summary>
    /// <param name="source1">First string</param>
    /// <param name="source2">Second string</param>
    /// <returns></returns>
    public static int Calculate(string source1, string source2) //O(n*m)
    {
        var source1Length = source1.Length;
        var source2Length = source2.Length;

        var matrix = new int[source1Length + 1, source2Length + 1];

        // First calculation, if one entry is empty return full length
        if (source1Length == 0)
            return source2Length;

        if (source2Length == 0)
            return source1Length;

        // Initialization of matrix with row size source1Length and columns size source2Length
        for (var i = 0; i <= source1Length; matrix[i, 0] = i++){}
        for (var j = 0; j <= source2Length; matrix[0, j] = j++){}

        // Calculate rows and collumns distances
        for (var i = 1; i <= source1Length; i++)
        {
            for (var j = 1; j <= source2Length; j++)
            {
                var cost = (source2[j - 1] == source1[i - 1]) ? 0 : 1;

                matrix[i, j] = Math.Min(
                    Math.Min(matrix[i - 1, j] + 1, matrix[i, j - 1] + 1),
                    matrix[i - 1, j - 1] + cost);
            }
        }
        // return result
        return matrix[source1Length, source2Length];
    }
}

The below script needs to be able to find the current output cell; this is an easy method to get it.

# Let's build a form scanner with notebooks and OpenCV

We'll build a form scanner with OpenCV using classic computer vision algorithms as well as a bit ML/Deep Learning
*Of course*, everything will be in C#. We'll also take a quick look into C# notebooks


## Topics
- OpenCV & C# Notebooks
- ML/Deep Learning vs. Klassische Algorithmen
- Form scanner with working ***Live-Code 🚀***
  - Scan document (Perspective Transform)
  - Segmentation of document
  - Text recognition

<img src="https://upload.wikimedia.org/wikipedia/commons/5/53/OpenCV_Logo_with_text.png" width="100"
     style="display: block; margin-left: auto; margin-right: auto" />

# OpenCV

- OpenSource
- De-facto "standard" library for Computer Vision
- C++ library with Wrappers for multiple languages

- Appache-2.0 License

<img src="https://socialify.git.ci/shimat/opencvsharp/image?description=1&forks=1&language=1&owner=1&pattern=Plus&stargazers=1&theme=Light" style="height: 280px;" />

https://github.com/shimat/opencvsharp - Apache-2.0 License

# C# Notebook

<img src="https://user-images.githubusercontent.com/2546640/94438730-833fed80-016d-11eb-94e6-da7b51abf58a.gif" />

# C# Notebooks
- Kernel also runs in Jupyter
- Jupyter Extensions for Presentation

In [2]:
Console.WriteLine("Hello .NET Day 2022");

Hello .NET Day 2022


## Dependencies
- Nuget and Usings just work as you would expect

In [31]:
#r "nuget: OpenCvSharp4.Windows"
#r "nuget: SharpCompress"
#r "nuget: Microsoft.DotNet.Interactive.ExtensionLab,*-*"

Installed Packages Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22405.1 OpenCvSharp4.Windows, 4.6.0.20220608 SharpCompress, 0.32.2

# Let's test it

In [4]:
using OpenCvSharp;
Mat src = new Mat("20220831_124447.jpg");
double aspectRatio = src.Width / (double)src.Height;
Cv2.Resize(src, src, new Size(2048 * aspectRatio, 2048), interpolation: InterpolationFlags.Area) 

In [5]:
// src // Crashes the kernel in jupyter (incorrect native memory access)
src.Size()

Width,Height
1536,2048


# Formatter

- Notebooks have "formatters"
- Last Statement without `;` or `.Display()`
- Output: HTML helper [`PocketView`](https://github.com/dotnet/interactive/blob/main/docs/pocketview.md) - simple syntax using `dynamic`

In [6]:
#pragma warning disable CS1701
using System;
using System.Threading.Tasks;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.Formatting;
using OpenCvSharp;

using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

private static PocketView CreateImgTag(byte[] data, string id, int height, int width)
{
    var imageSource = $"data:image/png;base64, {Convert.ToBase64String(data)}";
    PocketView imgTag = img[id: id, src: imageSource, height: height, width: width]();
    return imgTag;
}

Formatter.Register<Mat>((openCvImage, writer) =>
{
    if(openCvImage.Height > 640 || openCvImage.Width > 640)
    {
        double aspectRatioWoverH = openCvImage.Width / (double)openCvImage.Height;

        double newWidth, newHeight;
        if(openCvImage.Height > openCvImage.Width) {
            newHeight = 640;
            newWidth = 640 * aspectRatioWoverH;
        }
        else {
            newHeight = 640 / aspectRatioWoverH;
            newWidth = 640;
        }

        Mat resized = new Mat();
        Cv2.Resize(openCvImage, resized, new Size(newWidth, newHeight));
        openCvImage = resized;
    }
    var id = Guid.NewGuid().ToString("N");
    var data = openCvImage.ImEncode(".png");
    var imgTag = CreateImgTag(data, id, openCvImage.Height, openCvImage.Width);
    writer.Write(imgTag);
}, HtmlFormatter.MimeType);

In [7]:
     src.Display();

# The Goal

```json
{
    "First name": "Alex", "Company": "Noser",
    "Interests": {
        ".NET": true, "Computer Vision": true, "Machine Learning": true,
        "Performance": true, "Testing": false
    },
    "Lunch Preferences": {
        "Vegetarian": true, "Meat": false, "Vegan": false
    }
}
```

# Possible Solutions

- End-To-End ML Model
  - *In theory*, Deep Learning promises to allow End-To-End solutions - not viable in this showcase though
- Combination of classical CV-Algos & Deep Learning
- Classical CV-Algos

# ML / Deep Learning

- State-of-the-art Computer Vision
- Idea
  - Raw-Data plus Result => "Meta algo trains an algo"
  - New samples can be trained later
- Usually CNNs, Convolutional Neuronal Networks:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Typical_cnn.png/800px-Typical_cnn.png" />



# ML / Deep Learning

- Often requires lots and lots of data
  - Data Augmentation
  - Transfer Learning
  - GANs / Generate artifical training data
  - ...
- Training often resource intensive

# Classic CV-Algorithms
- Edge / Contourdetection
- *Primitives* such as Erode / Dilate
- Thresholding e.g. [Otsu](https://cw.fel.cvut.cz/b201/_media/courses/a6m33bio/otsu.pdf)
- Surf, Sift, HoG ...

# Classic CV-Algorithms

- Manually select and tune algorithms and their parameters
  - Only few samples necessary (Validation/Verification is still necessary!)
- Usually much less computational resource required

# Classic Algorithms vs. ML / Deep Learning
## Question to ask:
- Available Data: How much? In what quality? Effort to label?
- Environment: Controlled? "In-the-wild"?
- Explainability: Do decisions need to be justified?

- Lifecycle of Product?
  - New Data? Drift of data?
- Target Platform
  - Cloud vs. Edge?
  - Training vs. Inference?
...

# Classic algorithm vs ML / Deep Learning

  - **One** Picture

- **In-the-wild** but only one picture ;)

- C# Notebooks

- **45 minutes**

*=> Classical Algorithms / Combination

# Tasks

1. Dewarp picture
1. Text recognition
1. Checkboxes & Content
1. Association between labels & data

# 1. Dewarp picture

- "Classical Computer Vision Task"
- Idea: Find largest quadrangle

In [8]:
Mat grey = src.CvtColor(ColorConversionCodes.BGR2GRAY);
grey.Display();

In [9]:
Cv2.Blur(grey, grey, new Size(12,12));
grey.Display()

In [10]:
Mat thresholded = grey.Threshold(127,255, ThresholdTypes.Otsu);

Mat kernel = Cv2.GetStructuringElement(MorphShapes.Rect, new Size(40,40));

Cv2.Erode(thresholded, thresholded, kernel);
Cv2.Dilate(thresholded, thresholded, kernel);

thresholded.Display()

<img id="307103323d7e4cc1bc84fd4dc031a250" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAeAAAAKACAAAAABho/SOAAAUKUlEQVR4Ae3Bv49cV5rf4c8XmoAElgo4gFpdlZOAs13AVOgxIAlYoEqZAzIQnTkfMpXqlulQnL9hqUDMWRWNBHgdDg1oNmXndUkOIAbDYDqw8LpHQ0nNZnV33fPj3nNPvc8jXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5pwVROuasJVTbiqCVc14aomXNWEq5rIyzhjSovrj8hqM2Gbx9+scb0QWW0mnOP1HxpcfiIr40Jr5risRFbG5dopLhuRlbGjaYvLQeR060908epfn+LSEjmtZnS0bHApiZyMAGvmuFRETkao41dTXAIiJyPO9NUxLorI6MYzEpivccFERv/7d6QhXCCRkZHSeo7rTGRkJPbqP73EdSIyMvKYtrgdiXxmK7KZf3eM24HIZ7YiqyUN7hIiHyO746u4C4l8jB68fs0Udy6RzWFLj6YtbguRzWFLv+bfHePOENkYQ1jS4H4lsjGGsp7j3hDZGANqp7i/E9kYg5u+fs2eE9kYRZj/5Sl7TGRjlOLxUcO+EtkYJTk6mrOPRC6bCaU5fjVl34hcXhxQqOnLH9kbIhejZPM1+0HkYhRuSUP9RCarGWOwnlM3kcmTOSPx6nhKvUQmxshMW2okMjHG58HTNbURmRjj9O//p6EmIhNjxNYPnlIJkceiYdx+fDmlBiIPowrTlpETeRi1uPvqL08ZL5GHUZfls8eMkshitqI+3z6/y+iILH64TqXaKaMisjCqNm0ZC5GFUb0HT9eMgMjC2AtLGgoncjhs2Rsv/++cgokcXhywV9oppRI5GHto2lIgkYOxp+6+WlMWkcG9r9hny4ZyiAw2E/bc0dGcMogMDHeinTI8kYHhfvbwDy1DEhkY7rT50RFDERkY7qwlDUMQGRhum2+f36VvIr0nczqafn/AnmiZ0ieRntHR/YcY++Tudy09EcndeEZHzRJj3zz+Zk0PRHLNgq7EL4x9svzmiLxEcouGrsQ7vrnNfjj+bk5GIrlFQ1effstZi4b90U7JRST31T0CiJ9cv8I//P4eJ1reO2BfTFvSE8kZIcRP/nSL08T1H9gjc9akJZIzQoifGG8RJ4w9s2xIRqQ2WxFC/MR4izhh7J+jozlJiNSMcF/fNd4iThj7qWVKNJGakZA4seGNCftn+vo1MURihy0JidOe3WAPiRgiscmGhMR5NhP2hIghEjtsSejpR5xjM2FPiBgisRcHZCDO2kzYEyKGSMzI4+oxb/n+gGvX2AcihkjMyEhcZDWjRiKGSMzISFzIqJGIIdJ69DkZiQsZNRIxRFqbCb25/5C3TYB/+4TKiBgirc2EHk1b3rXhlOtXGDsRQ6S1mdAzcZHVjLETMURamwkDEu8yRk7EEGkZQxLvMkZOxBBpGUP687+w1YsDRqtZEkMk9WTO8L79lK02/GzCeNx/SAyR1GpGWcQ2f/yE0RBRRFJGYW4esY0xGiKKSMoozHzNRTYTCreeE0UkZXTUTsir5bu7XGC2omi/fUUUkZTRmThhZPTwD7zx+jXbGaUScURSRmfihNGLx3e4kFGa46vEESk1CzoTpzS3b5DX/Yecr+Hv/ukexWinxBEpLRo6E2+brchLXMYohogkUjK6E+8w+iC2MwoiIomUjO7EKYuG3nz4kq2MgohIIqHZiu7EKYuG3s35uzU/M8rx7adEEgl9c5vuxCnNgoGInxnlmLZEEgkZAcQpi4aB/PYVbxjlELFEQkYAccqiYVgCoxwilkjICCBOWTQM69v/vqEgIpZI57AlgDhl0eB+NV8TS6Qz2RBAnNIscL+6eUQskY4RQvzisMWdJqKJdIwQ4heHLe40EU2kY4QQvzhscac0S6KJdIyO1pyY84vDFneKiCfSMToSb5tscKeIeCIdo6s1c045bHGniHgimc2EAK+Op/zssMX9avUZ8UQyLw4IN6WFwxb3q6/vEk8kY8S6+wj3K5GASMZwSYkERCpf/E9cSu2UBEQqT+a4lEQKIhXDJSVSEKkYLimRgkjFcEmJFEQiiwaX0uQ5KYhEFg0uJZGESMRwSYkkRCKGS2m+JgmRiOFSunlEEiKN2QqXkkhDpPHDdVxKIg2RhuFSapakIdIwirGeMX4iEZGGUQzBasbIiUREEtd/oBgCZitG7elHJCKSOGwphjhhjNpvX5GISMIohzhhjJpIRSRhlEOcMEZNpCKSMMohThhj1k5JRSRhlEOcMMZMJCOSMMohThhjJpIRKWwmFKJZ8hNjzEQyIgWjFM2Sn1x/NGO0Js9JRqRglGLZ8ItmwTiJdEQKRimaJWcYoyPSESkYpWiWvMMYGZGOSODJnFI0S85hjMV8TToigdWMMhxf5ULPbjACHz0lHZHAakYZxKWu3VtQOJGQSODFAWUQO1g0lE0kJBIwCiF2sGgom0hIxLv2VwohdrBoKFqzJCERb9HQ1dNb5CB2sGgomkhJxDM6m68bFiQndtAsKJpIScQzAq3/28ecuPUFiYgdLBpKtp6TkohnRGn/x4o0fvMjO1g0lEwkJaLNVkQTJzYT4ohd3PtfVyiYSEpEm62I1045YUQRuzHK1U5JSkQzElk2DScWhBK7ufcVxRJpiWhGQk//Mmf2xS2CiPOsOGVGwURaIpqRVAvTCR8/ojtxHmMsRFoi1mFLFvO10ZE4jzEWIi0R67ClEE8/4jzGSEyek5aIZZRCnGaMkUhMxDJKIU4zxkgkJmIZpRCnGSM0X5OYiGWUQpxmjJBITcQySiFOM0ZIpCYibSaUQpxmjJBITUTaTCiFOM0Yn2ZJaiKSUQxxmjE+zZLURCSjEC1TTjPGRyQn4tz6E4UQbzPGRyQn4qxmFEK8zRid9ZzkRJwXBxRCvM0YHZGeiGOUQrzNGB2RnohjlEKc9f3BhDFpp6Qn4hilEFsYIyIyEFGaBYUQ/7Ca8aujG4yIyEBEeXaDIjz4kjdWM8ZKZCCiGGWYr3njyZyR+uf/IAMRxSjDfM0bT+aM1OQ5GYgYBy8ow3zNG6sZIyVyEDFmK8owX/PGasZIiRxEjD/dogzzNW+sZozTgy/JQcQwCjFf88aTOeN09ZgcRAyjEOJnT+aMk8hCRLjyNwohfraaMUpHN8lCRJhsKIb4h9WMUVo2ZCEiGOU4vspPVjNG6eYRWYgIRlla4PoVRknkISIYLpHjq+QhIhgukfWcPEQEwyUiMhHhHn2OS0RkIsJtJrhERCYi3GaCS6OdkokIZ7hERC4inOESEbmIcIZLROQigj2Z49L45/8gFxHsyRyXhshGBFvNcGmIbEQww6UxX5ONCGa4NOZrshHBDJeGyEeE+uoeLg2RjwhluDSaJfmIUIZLQ2QkAs1WuDRERiLQbIVLYvUZGYlAhktD5CQCGS4NkZMIZLg0RE4izGGLS6KdkpMIc9jikhBZiTCGS0NkJcIYLg2RlQhjuCSmLVmJMIZL4uoxWYkwhktC5CWCbCa4JEReIshmgkthviYvEcRwSYjMRBDDJSEyE0EMl8LD+2QmQlz/AZeCyE2EWM1wKYjcRIhnN3AJHN0kNxHCcCms5+QmQhguBZGdCGG4FER2IsCiwaUgshMBmgUugXZKdiKA4VIQ+YkAhktB5CcCGC4FkZ/obrbCJTB5Tn6iu9kKl8D7r8lPdGe4F

In [11]:
Point[][] contours = Cv2.FindContoursAsArray(thresholded, RetrievalModes.List, ContourApproximationModes.ApproxSimple);

Mat tmp = src.Clone();
Cv2.DrawContours(tmp, contours, -1, new Scalar(0,0,255), 3);
tmp.Display();

In [12]:
Point[]  documentRect = null;
int maxLength = 0;
for(int i = 0; i < contours.Length; i++)
{
    Point[] contour = contours[i];
    double length = Cv2.ArcLength(contour, true);
    Point[] approximatedPolygon = Cv2.ApproxPolyDP(contour, 0.15 * length, true);

    if(approximatedPolygon.Length == 4)
    {
        double actualLength = Cv2.ArcLength(approximatedPolygon, true);

        if(actualLength > maxLength)
        {
             documentRect = approximatedPolygon;
        }
    }
}

documentRect = documentRect.OrderBy(p => p.X).ToArray();
documentRect = documentRect.Take(2).OrderBy(p => p.Y).Concat(documentRect.Skip(2).OrderBy(p => p.Y)).ToArray();

Point2f topLeft = documentRect[0];
Point2f bottomLeft = documentRect[1];
Point2f topRight = documentRect[2];
Point2f bottomRight = documentRect[3];

(topLeft, bottomLeft, topRight, bottomRight).Display();
int targetWidth = (int)Math.Max(topRight.X - topLeft.X, bottomRight.X - bottomLeft.X);
int targetHeight = (int)Math.Max(bottomLeft.Y - topLeft.Y, bottomRight.Y - topRight.Y);

Point2f[] targetPoints = new[] {new Point2f(0,0), new Point2f(0, targetHeight), new Point2f(targetWidth, 0), new Point2f(targetWidth, targetHeight)};

Item1,Item2,Item3,Item4
"{ (x:303 y:287): X: 303, Y: 287 }","{ (x:55 y:1672): X: 55, Y: 1672 }","{ (x:1336 y:515): X: 1336, Y: 515 }","{ (x:992 y:1873): X: 992, Y: 1873 }"


In [13]:
Point2f[] targetPoints = new[] {new Point2f(0,0), new Point2f(0, targetHeight), new Point2f(targetWidth, 0), new Point2f(targetWidth, targetHeight)};

Mat transformation = Cv2.GetPerspectiveTransform(new[]{topLeft, bottomLeft, topRight, bottomRight}, targetPoints);

Mat corrected = src.Clone();
Cv2.WarpPerspective(src, corrected, transformation, new Size(targetWidth, targetHeight));

corrected.Display();

# 2. Text Recognition / OCR

- Lots of training material (and easily generated) -> Deep-Learning
- Open-Source OCR: *Tesseract* by Google
- Preprocessing: Thresholding (Tesseract perfoms Otsu internally)

In [14]:
Mat cropped = corrected.SubMat(0, 650, 0, corrected.Width);
Mat greyscale = cropped.CvtColor(ColorConversionCodes.BGR2GRAY);
Mat threshold = greyscale.Threshold(127,255, ThresholdTypes.Otsu);
threshold.Display();

<img id="8a4169a36e764595a5bfcd3de9d7ed42" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAoAAAAGSCAAAAABWUC/yAAAgAElEQVR4Ae3BC2DP9f748edriGjYhi+ikNvK1GZfRB2rJulYsTBREadS6aKkLLqoULroylGUyiXKt/gudjaKQ0I2pqLILXLLl/gKubz+38+GQ79OOZ2PfT7n/3k/HqIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaIYhnNEMQzniGIYzhHFMJwjimE4RxTDcI4ohuEcUQzDOaKcRsLlsyjSeYoSIRyjHT7kBMsaUyR1FhG30aAvp2b2ZCJih/DbZuY+y39GUE5R75oPY/w3RDl9vkyAy2ZTSLhlNCAlZ19K83MmN18gy28hIu9gMyzv1qXQyniaEbF6B9/X4I+FE1eXSIaFoPwmYVoa/xFBOUWSmMe/LATOq8Qx29cQfT7HLSShLMaviHL6JC+Bct+cjUW4ZTQglbdxkqpblRO1zkWJ2PiPXpW38ccWN70hMx6+2txa+U19R0xLg7VpHZ7gFAnKKZLEPI5ptvcr4OyNHHXDJz9QtvbN91OoEV9Rt/SXFGlyACjzBYYop5GUHnMDikWY3AmQyts4SbUtyola56IUempg+aW1+SNCoD1/bGV8zzGcIkE5RZKYx1Hd305JgiMjGDCEiLmtaHQlvBV69Q4gcyj3RvH2j2WW1Qfe7wT3sXQ2UzridaKcPslLSu+PDg98gghhb1lAKm/jJNW2KCcRlCJPDprSkYgD+4DSZwL7y/BrgnIKVsbf8A6nSFBOkSTmcdSHV0RjEX4+E7j75fktiCi7750bICbYkogRfVGg/ZzNZYB9FX9Zdy4eJ8rpk7yk9P7ZVzRYSYRwy2hAKm/jJNW2KCd6/DGUIivjUdhw77p8oH6LN+mel/IyhcI3NXmYQsKYnhyVXvs5LFMmDkrEks5Upk5/E9LDOXSAl2rw9EIKTcWS/wQ81piTCcopksQ8fkUYMASQGkvjiBjeH+WY1fVQ/qXrxDYz8ThRTiMpvZ86a//RGhCUCKm8jZNU26KcqHUuylGCgtAu6qM0gkxLQ2osi8UiPP4IhRY3veYjjqq9bkkSETe+ixIx+raL8hk2YFoaQhEN19tCoZG9Ifj0PCL+MrsEICzYmwqdJiMo7FiQRsTNN6UQEX/+B19MH0y10e0oEmT87DHtkMQ8fmVOyoAhEBfqPZJCwiI/xwhXzeC4rhNRPE6U06d1bun97KgU9Y8rQFAiJKoSlq0cVXWrcqJdMShHCT3HsI0qRAx6ssIuhHktiXjtTpQiBy/6usroayl01ysxISIElIjRt73VnWEDpqWxbXXLri8AVfjpABE+fj6T0bexFeg0t/I2QDjj8D0PXh83GUFhbit49wYo8VlTIH7lzqoHiLjzFSx/G0NElfVnJubxK3NSBgyB2J1KEaHddI4RrprBcbsroHicKKdP8pLS++HJQZ0mg3DLaEAocvNYilTbopyodS7KUULHKRz1+cUo/Yff8A4Rz/X7IJ1jrplO7c7DiMi5EgWefgje7QZI1LJGtJ05LQ1Wxvccw3Gjb3sqE6T00oZEXD/p5T4g8OAwIoJpKHw/oXkroNVcFIhfyY3XdGRnk7W/lALue6F+zwd/GvUQJObxK8KnrQikoxR5dHC76RwjKP/SdSKKx4ly+iQvKb0fEDZVR1AipPI2TlJti3Ki1rkoR8279K6XOEZQEJQIKTGjNcftTfp5I9/UKAu11m+tArXX1VnTcQpsOLfkQRg2YFoarIzvOYZjclvf+DY0Xq4Umth10GAQyn57NhZBOU7YXBXiV/oXEZEyB4URfVEigtdoYh4n21INhUD68H4UCcW1m85RI/qiHHfk2uC6c/E4UU6f5CWl9wMfta+09ixBiZDK2zhJtS3KiVrnohwlKJbDKcA8FCZ0GzIAbhgfE+JEOxtsp9FyWJp4x6tQe92Bv9ScAs0WzWsJtddNS4OV8T3HcIzwTX1ovFwpNLErCsLyRhQSlOOEnmMgfqViWd4YhRF9Bw/C8lnLxDxO1v3tSRkQSK+1liKhuHbTOarO2sGDOO7ncsnzSuNxopw+yUtK7yfi6hmv3CkoEVJ5GyeptkU5Uevckb05SlDYGehFEYVvGl7yTzh/hfIrH3bgkxTyky6fxewrBj7RduZP5Wm2SAH/F9PSYGV8zzEc1S7r6iyg8XKOUxBmXEUhQbHsmwj0oucYiF+pFBIUrp80uROFJDGPk7x+a9VlVSCQjlIkFNduOkUC6dG7OW7F+Qzvh9eJcvokLym9H4ugghIhlbdxkmpblBO1zkU5SlAQUCIEBS6Zf0QWtLj/WX5tfa3rJ0CTPGX2FYMGt52Jfplw69+BYQOmpcHK+J5jOErOW01E4+Uc83IfEK7IpZCgRAhFFIhfqRQSFITJnSgkiXmc6O+9USIC6cP7USQU1246hRY1Q/kXYcL1eJ4op0/yktL7sdz5WtNFKBFSeRsnqbZFOVHrXJQiTw76uC2TrkexCApcMv/msQta3P8sv7a+VpeJ0CTvtlGCgqBfJtz6d8D/xbQ0WBnfcwyFll28b0pHIhovV04gzLiKQoLCN013l38B6EXPMWy7aLNSSFAQJneikCTmcSIpsbIuEYF0lCKhuHbTsbzdnc+bcdzjj535M4Yop8GGDVwCJC8pvR/Lp6mHQYmQyts4SbUtyola5+ZeQaEv/NV+gPteQInYWBMlQthSFeX/WF+ry0RAzph1afJiiF/5/H0oEbXXTUuDlfE9x1BoeP8hA7A0Xq6cQLgil0KCgjDqNiKEnmMgfqVSSFAQJneikCTm8S/zbt25mSJxIaWI0P9pIuZfEr20DseN7VXtBwxEOQ0eexwFkpeU3k+h1fVAiZDK2zhJtS3KiVrnolgarD34XR3YXgUlovKPKBFDM31bP27LcR3fp9B5a7pMBARYWwve71R5O0rEsAHT0mBlfM8xWPq8yo5YLI2XJ+bxL8IVuRQSFATFImyuSn7zX5RCgsLDQyZ3opAk5nHckIe5bRRF4kJKEUGJ+DBd15/DcYMfZWsVDEQ5DR57HAWSl5TeT5Gmi1EihGOWXoil2hblRK1zGcbLdz0EvTtcScTfxjRNh2d/BCVibC/4Op5jbh/FsFL3rZ2cMytjEhGv9uHc/Bh4vxMsbEqEMC0NtiduGvoQsD/5qy0+iqTO6joey3MpTUC4IhfLrFQUhE9bAZ9cTs8xrG+yQykkKIzoixLx9UUHE/M45oX7KvccxlGDH224Asvw/igwoRufN+O45/r58qpjgCinkW8rx6xstZWIl5/kqK0UarR9KyfxUei16ziq7fw9lIrZWvngLixRevFn/IuPbUSUq5+HZWwvLp9FRIXdJRcmEXHW3mlpQLssInRnLEVG3cboh39kaTU6zeXlPiBc9z6FBIWYXXc/DA13wj0jOHjBKqWQoIBwWW7U4R0+SMzjqAOVwsq/lPs5+FcgFBc1/Wo4fGn+npIcN+r26N0YFlFcb/1yYltwYJcPS5TOa8mJgkTUr0+RIJWaEzHzUHQrLKtX+

In [15]:
List<(Rect rect, string text)> detectedText = new();

using(var tesseract = OpenCvSharp.Text.OCRTesseract.Create(Environment.CurrentDirectory, "eng"))
{
    tesseract.Run(threshold, out var outputText, out var componentRects, out var componentTexts, out var componentConfidences);
    
    Mat copy = cropped.Clone();
    foreach(var (rect, index) in componentRects.Select((r,i) => (r,i)))
    {
        string text = componentTexts[index];

        if(!string.IsNullOrWhiteSpace(text))
        {
            Cv2.Rectangle(copy, rect, color: new Scalar(0,0,255));
            Cv2.PutText(copy, text, new Point(rect.Left + 100, rect.Top - 10), HersheyFonts.HersheyPlain, fontScale: 2, color: new Scalar(255,0,0));
            detectedText.Add((rect, text));
        }
    }
 
    display(copy);
}

# 3. Recognize checkboxes

- Same idea as used for dewarping
- Find *Quadratic* contours i.e. squares

In [16]:
Point[][] contours = threshold.FindContoursAsArray(RetrievalModes.Tree, ContourApproximationModes.ApproxSimple);

List<Rect> checkBoxes = new();

for(int i = 0; i < contours.Length; i++)
{
    double length = Cv2.ArcLength(contours[i], true);
    Point[] approxPoly = Cv2.ApproxPolyDP(contours[i], 0.14 * length, true);
    Rect boundingRect = Cv2.BoundingRect(approxPoly);
    double area = boundingRect.Width * boundingRect.Height;

    if(approxPoly.Length == 4 && Math.Abs(1 - boundingRect.Width / (double)boundingRect.Height) < 0.1 && area > 120)
    {
        checkBoxes.Add(boundingRect);
    }
}

Mat copy = cropped.Clone();

foreach(Rect checkBox in checkBoxes)
{
    copy.Rectangle(checkBox, new Scalar(0,0,255), 2);
}

copy.Display();

In [17]:
using OpenCvSharp.Dnn;
CvDnn.NMSBoxes(checkBoxes, checkBoxes.Select(_ => 1f), 0.8f, 0.3f, out int[] indices);

copy = cropped.Clone();
checkBoxes = checkBoxes.OrderBy(c => c.Top).ToList();
foreach(int i in indices)
{
    copy.Rectangle(checkBoxes[i], new Scalar(0,0,255), 1);
}

copy.Display();

In [18]:
List<(Rect location, bool isChecked, Mat roi)> checkBoxesWithValue = new();

foreach(int i in indices)
{
    Rect rect = checkBoxes[i];
    rect.Inflate(-(int)(0.2 * rect.Width), -(int)(0.2 * rect.Height));
    Mat roi = threshold.SubMat(rect);
    bool isChecked = roi.CountNonZero() / (double)roi.Total() > 0.9 ? false : true;
    checkBoxesWithValue.Add((rect, isChecked, roi));
}

checkBoxesWithValue.Select(l => (l.location.Left, l.location.Top, l.isChecked, l.roi)).Display();

index,Item1,Item2,Item3,Item4
0,404,285,True,
1,406,286,True,
2,404,308,True,
3,404,331,True,
4,404,354,True,
5,404,377,False,
6,404,405,True,
7,406,429,False,


# 4. Association of data

Idea
- Label & value are on the same row => find table rows
- Fuzzy-search each label (Levenshtein-Distance)
- Search on the same line for the closest value (Text / Checkbox)

# 4.1 Line detection

In [19]:
Mat lines = threshold.Clone();
Cv2.BitwiseNot(lines, lines);
lines.Display()

<img id="128b0464b20c4d7a8ce72db219f9695b" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAoAAAAGSCAAAAABWUC/yAAAgAElEQVR4Ae3BC2CP9f7A8ffHXX7I5jIlIUJlbG6hqPycxYP6KY5LhVbSRakoJCqitFLpgg6SMKOR7SSxmn/ufrYyd7nl+sM2TGvCvv/fs+HQ6ZTTeex5znm+r5egaTYSNM1GgqbZSNA0GwmaZiNB02wkaJqNBE2zkaBpNhI0zUaCptlI0DQbCZpmI0HTbCRomo0ETbORoGk2EjTNRoKm2UjQNBsJmmYjQdNsJGiajQRNs5GgaTYSNM1GgqbZSNA0GwmaZiNB02wkaJqNBE2zkaBpNhI0zUaCptlI0DQbCZpmI0HTbCRomo0ETbORoGk2EjTNRoKm2UjQNBsJmmYjQdNsJGiajQRNs5GgaTYSNM1GgqbZSNA0GwmaZiNB02wkaJqNBE2zkaBpNhI0zUaCptlI0DQbCZpmI0HTbCRomo0ETbORoGk2EjTNRoKm2UjQNBsJmmYjQdNsJGiajQRNs5GgaTYSNM1GgqbZSNA0GwmaZiNB02wkaJqNBE2zkaBpNhI0zUaCptlI0DQbCZpmI0HTbCRomo0ETbORcDkpvm5NvrjOQpDiHJl7Nxeov458i1sTNIEtY7k0d3QhKGMIv+3O1gP59yiESzR+z6to/wnhMropDb5uTR7FR30Adar1t6z8scvKZqreRwRFFl2F6b4fyFNnE6sIuq481+zlj5X6ruYZPzQF4TcpOibwb1EIl0ilRvIPTYHtRzinQg2yNnJeU9Ky0X5FuIz8DeGn2vswKSY+AqjDFbnIgTDhQou8CEFV/jLpcEX+WOPVn47aBDdWXiT8prH9OyRC9YS5L3KJFMIlUqmRnLOq1I3Aviqc9eltV5O9c8qb5FnPjfxw8ibyrS0O5DRCEy4ndTL6UwSTostsQB2uyEUOhAkXWuRFyPPCyOMNdvJHFL55/LHamydHc4kUwiVSqZGcNfWB5BQo1J/RQwhquYT1X0GvkMc/AEYN5u1cHiifU38rcO9seIsGd9B5Dm4nXEb+hjklszwjXyRIUSobUIcrcpEDYcJFFEK+oSM6zyGoeEng5M9AiRx+TSFcgjqbPr2fS6QQLpFKjeSsuxefwKS44mfg3X4tlhOUXfK+6ZBpLCeo/1gEmNeqcg5Q8mixartxOeEy8jc8WeL2r7fUIUgx8RFAHa7IRQ6ECRca/hJCvjqbEKj6drUIYOvy3kyNTO5HHs8na18ljyJ6MmfF73wWU+duI1IxxdOJTh16Q7ynDXPhyb0835Q8nTA1GAYvreNiCuESqdRIfkUxegig9tbPIGjAGwjn1NyG8A8zui28E5cTLid1sgQ7qv9lEaAQgtThilzkQJhwoa/aIJylEFAk5t6VQHs6JqD21s/ApBj+Cnkar55/F2ftrNYwhaBp9yEE9ZnwXQSDRndMQJFPPNvCyNN3ArR//haClrQ+AyialVoMs7ugEAhtlkDQlE+SCdq08Z5GHYZxoE8i+drT447oRFRqJL/SKnn0EEgPGf8oeRRN1nCO4su2nDejG4LLCZfRIm9OSUKP5P4lCRRCkMo9gqkSZx0IEy50ZSbCWYrJ0VTkEEEjhh67EsUtywh67H2EfEW+v+FQn8/JM+6JjFCCFAhBfSb0/IRBozskUrHmshlPA4coW5ygAFf8TJ8JVAJmtzxcEVD8Uvid12emd0Eh0HIJ3PcpnG6+BthYt9zB4gS9/wSmv0UTdOjan1Mj+ZVWyaOHQHqIkE+R2IFzFF+25bwyxxBcTriM/A1PloChI2Z3AcVHfQBFvikPku9AmHChRV6EsxRzOnPWzSsQxgz89H6Cno3pNJdz5ndgZ9wggtp8hQDPvwb3TQdUbv31LLizYwLU2TQ5mvP6THhhFKiTDTYTNLNrv/dAweuDCGqfgMA13VcuAZa0RICNdZk2fw7l1lYvehp46+mtk18v2/c1SI3kVxS3LeHuuQj5Xh6W2IFzFMI/zOiG4HLCZeRveLIEoLh6PwohSB2uyEUOhAkXWuRFOOuWb8c9yTkKAYUQpM60XcR5pVKuqELtvdmw69pKh2BntR015nSGqrtPF4VBozskQu3Nk6M5x7vok56wrp6Qp9uMEcNAkX39PkwK4TxFWAA21l3ThKDkVgj0H4sQ1H6+pEZysbADCPjiB8aQLyQ9oSNn9R+LcJ7Mb19tNy4nXEb+hjklgbvmHan2k0IIUocrcpEDYcKFvmqDcJZCMBVOBm5BoPv0IaPh0x6ZIVyo3JYKrK8HDVLffwJ2Viu+ZG9nWNn0lmWws1rHBKizaXI05yhqb4V19YQ83WYgoKi3njwK4TzF5GjYWFcw1VuHQP+xw0Zgar4sNZKLTX2g6yzwxe+sQb6Q9MQOnLW9xosjOa9ktv+Wk7iccBn5G+aUJOiLtk+8rxCC1OGKXORAmHChRd5Hx3OWQqCcbxL5BGpvXnorbKwr/Mpd87g9mQapX7fmjqQRwxbcWfY4K5sKsKZRxwSovXlyNGclGl8YwLp6nCegaPsleRSCqUR3YBKTo2FjXSGPQmBGty6zyaNSI7nIQx8drH8IfPEI+ULSEzqSzxd/vCzn1d3IwBjcTriM/A1PlsCkEIUQpA5X5CIHwoQLLfIinKUQUCAEKQRY2qKQarb8zQH82rW7ZnaHtZHCHUkjhi24E7kpbeIjwKDRHROgzqbJ0ZylttckaF09zun3HiiSvORRCEGKfAJsqiPkUQgouswmj0qN5EKPjEcI8sUPjCFfSHpiB/I0WYXwD4ruM3E94TLyN8wpien9x1Y3QQhShytykQNhwoUWeRHyDR3RbgFdZyKYFAIsbTHlwWbL3xzAr1XdPbM7rI2c0FchoJCb0iY+Aqxp1DEBam+eHE2e+itKdp5D0Lp6wgUUbb8kj0Lg+jVljj8NTGJyNBW/qyzkUQgouswmj0qN5ELqTJ0fCPLFI+QLSU/oiOmBqdy8ivOGv5RdCk24HKpWZSngb5hTEtNti4qAEKQOV+QiB8KEC33VxptEnkZr9l8Nbz2NEFRlD0KQIuwgwj+puntmd0D90vpbf2PYVOfpsQhBO6t1TIA6myZHk2fgmCGjMa2rJ1xAkeQlj0JA0XcCQYrJ0bCxrpBHIaDoMps8KjWSf7hlYrnK5EsPEfIpxjxPUIulWQ12cF7vyQeuQkO4HF4ajgD+hidLkKfmNhCC1OGKXORAmHChRV4E05bqRa/bARUOIQQdLo8QNHhUoFK7BZw3517ybK8xszuggOq74N7ZhysgBA0a3SERam+eHI3pvccJzcC0rl5qJP+gSPKSRyGgEEyKsAARK4sJeRQCrw7pMps8KjWS84a8yoS+5EsPEfIphKC74+XaHzlv2MtUOoSGcDm8NBwB/A1PliDf6sYIQYpzGnyP6UCYcKFFXgbRb9xrMH7uVwT9LXp1PAwoD0JQ78lwwybO+bAvg34ZW71Lm9ax3Qh67H12R2TCvbOh6WqCFB0ToEJKlcGvASX8N4YFyLe49YwemJ79JgUUSV5MrRcjoLhtCXD710yO5tq1oUIehUD/sQhBN3xXNDWSc55+6/DkQZw17OVNN2AaOAYBuk/n5lWc92xMIHI/GgiXU6AS59RZUomgfkM5qxJ51leoxEUC5HnsM85a0KI0pzIrHS56JaZcWdGcfwhQkaCftkZienASSV6CjpU53TSFoCxPh0Qg0SBIymWQr+8E+oysQP2DzG5Jv/dA8dm95FEIZF757quwuRy805+iG2oJeRQCKL7x5hYODUBqJGcVP+IR/uGnK9r/HQhJz+3wBRT+NqL0ac7r+2FWGTST4HxVw8lYTrFyAUy5cssyLtSeoK1bydeeIysJurNI1hJMNeusCRDUfr7w2IdFlzQjT98JUOSbWzDdu3YXKIR8CoHQtMoETVn9oUDzZQh5FAJ0jmNxTsnWS2ISU

In [20]:
Mat horizontalStructure = Cv2.GetStructuringElement(MorphShapes.Rect, new Size(lines.Width / 20, 1));

Cv2.Erode(lines, lines, horizontalStructure);
lines.Display();

<img id="84d462e09f524e69ac751461bb352e60" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAoAAAAGSCAAAAABWUC/yAAAE8ElEQVR4Ae3BMaggBAGH8e9DG5wMvbFBjFpEGhxCoilcRKPDobGmApfIoNHBoSFE0MHFRRyiSRH3w2gQwSKHGropZxNUHCoE8Ti4d+/e87z0+X/P9/1+kgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJjnDb37jqfnKSJF+GZ7jiCW7Jr4GX/8V5JsmQJEOSDEkyJMmQJEOSDEkyJMmQJEOSDEkyJCfuwR8Cl94iuZGcEq/fx2d448cc6wIHvEvOFDnzHuWA17ipl77Jpy7/huxJDnmFAy5yrJ9w2AeXyC2S/J/u4rCP3+dY/+Y4d3OeSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMyYn77e9JjiGnzuO3P8fn8chDHPYrcsbIefCnO/nU2z/jWN/jgLfJV0RyxQMc8BeO9eZtXPOD/5AvRnIiLv6Im3rxLc49SYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYbkpP3uAjf4BckVcqq8wWHP/pGjPfANrvnrf8mZJGfVd2/nmsv/42h/54D7yGkjgac54NU/c6R7L3KdD14gX5jkS/X8T7mJu8k1kgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgzJSfvnPVz18R0k15NT4Jcc5Q8fcpyHHuM6/3iOnE3ytfbkw9zg4fc40rcvcNh7l8nJklz1rTs57MN3OMIr3+Eo3/+I3DLJCfv5/XyG95/iXJNkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJkSJIhSYYkGZJk6BPsTDuT8Y5veAAAAABJRU5ErkJggg==" height="402" width="640">

In [21]:
Mat largeHorizontalStructure = Cv2.GetStructuringElement(MorphShapes.Rect, new Size(lines.Width, 1));

Cv2.Dilate(lines, lines, largeHorizontalStructure);
lines.Display();

<img id="4a9a6b46f4164033b921933c351380ae" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAoAAAAGSCAAAAABWUC/yAAAEZElEQVR4Ae3BPYqeBQBG0XsrOwtLLbIJwVi7A4vsQbBTLKzEwiLgAgxuw8I6hYKSJmLlT5vWAUOY5lMQG0HnNc7MYz7uOZIMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IM+Zjkxjx9g38mf7h3l+dw7zWS/0KSIUmGJBmSZEiSIUmGJBmSZEiSIUmGJBmSZMgPyAvgPmfKEzl/bz/kkMsLbpknkj89+YajPv6W6+CJ5KZ88TlX8BWOeXSH5NpJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgx5Qf7nXuaMeSJn5tkDns+vH3LrPJEc885XHHLxM4d5Irlev/zAMa+D/I2XPiW5cZIMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IM+YC8aC7f5Wx4IufsTY74mhVPJN9zwJO3uH6eSK7T+xzz5Xf8Tv7iziOSWyPJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkCRDkgxJMiTJkJfkBfHRJ5wfT+Q8fcZh939ixRPJv3GXwx4/5SqeSG7Ij8+4ilzp1fdIbogkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ5IMSTIkyZAkQ78Bv+gzw6h9l1IAAAAASUVORK5CYII=" height="402" width="640">

In [22]:
LineSegmentPoint[] detectedLines = Cv2.HoughLinesP(lines, rho: 1, theta: Math.PI / 180 * 30, threshold: 10, minLineLength: lines.Width / 2, maxLineGap: 0);
Mat copy = cropped.Clone();

foreach(LineSegmentPoint line in detectedLines)
    copy.Line(line.P1, line.P2, new Scalar(0,0,255));

display(copy);

In [23]:
int[] lineSeperators = detectedLines.GroupBy(l => l.P1.Y / 20 * 20).Select(g => (int)g.Select(l => l.P1.Y).Average()).OrderBy(y => y).ToArray();
lineSeperators.Display();

int getLine(int y, int[] lineIndices)
{
    (bool match, int line) = lineIndices.Select((y_line, i) => (match: y_line > y, i)).FirstOrDefault(l => l.match);
    return match ? line : lineIndices.Length;
}

getLine(310, lineSeperators).Display();

index,value
0,223
1,248
2,275
3,395
4,469


3

# 4.2 Association of values

In [24]:
static Point Center(this Rect rect) => (rect.TopLeft + rect.BottomRight) * 0.5;

var textsWithLine = detectedText.Select(t => (line: getLine(t.rect.Center().Y, lineSeperators), t.rect, t.text)).ToList();
var textsByLine = textsWithLine.ToLookup(t => t.line, t => (t.rect, t.text));

var checkBoxesWithLine = checkBoxesWithValue.Select(t => (line: getLine(t.location.Top, lineSeperators), t.location, t.isChecked)).ToList();
var checkBoxesByLine = checkBoxesWithLine.ToLookup(t => t.line, t => (t.location, t.isChecked));


In [25]:
LevenshteinDistance.Calculate(".Net", "BNet").Display();
LevenshteinDistance.Calculate("Computer Vision", "DComputerVision").Display();

1

2

In [26]:
(int line, Rect rect, string text) getLine(string label) {
    return textsWithLine.MinBy(t => LevenshteinDistance.Calculate(label, t.text));
}

string getTextValue(string label)
{
    (int line, Rect rect, string actualLabel) = getLine(label);    
    return textsByLine[line].Where(x => x.text != actualLabel).MinBy(x => x.rect.Center().DistanceTo(rect.Center())).text;
}

bool getIsChecked(string label)
{
    (int line, Rect rect, string actualLabel) = getLine(label);
    return checkBoxesByLine[line].MinBy(x => x.location.TopLeft.DistanceTo(rect.TopLeft)).isChecked;
}

In [27]:
getTextValue("First name")

Alex

In [28]:
getIsChecked(".NET")

True

In [29]:
public record Interests(bool Net, bool ComputerVision, bool MachineLearning, bool Performance, bool Testing){}
public record LunchPreferences(bool Vegetarian, bool Meat, bool Vegan){}
public record ScanResult(string FirstName, string Company, Interests Interests, LunchPreferences Lunch){}

In [30]:
using System.Text.Json;

var result = new ScanResult(getTextValue("First name"), getTextValue("Company"),
               new Interests(getIsChecked(".NET"), getIsChecked("Computer Vision"), getIsChecked("Machine Learning"), getIsChecked("Performance"), getIsChecked("Testing")), 
               new LunchPreferences(getIsChecked("Vegetarian"), getIsChecked("Meat"), getIsChecked("Vegan")));

JsonSerializer.Serialize(result, new JsonSerializerOptions() { WriteIndented = true } ).Display();

{
  "FirstName": "Alex",
  "Company": "Moser",
  "Interests": {
    "Net": true,
    "ComputerVision": true,
    "MachineLearning": true,
    "Performance": true,
    "Testing": false
  },
  "Lunch": {
    "Vegetarian": true,
    "Meat": false,
    "Vegan": false
  }
}

# Conclusions 🚀

- Classical algorithms vs. Deep Learning = Manual fine tuning vs. lots of data
- Built an end-to-end scanner in about *one day*
- (C#) Notebooks should be in the toolkit of every dev - for explorative coding or presentations

- OpenCV(Sharp) & C# = ❤
- Combination of classic CV & Deep Learning = ❤
- Jupyter Notebooks & RISE/Reveal.js for a live code demo = ❤

### Questions?